In [18]:

import pandas as pd
import time
from collections import deque, defaultdict
from concurrent.futures import ProcessPoolExecutor
import dask.dataframe as dd
start_full = time.time()


In [19]:
seats_path='/home/it/Desktop/RUN/NEUT_Official/NEUT_R1/OfficialData/R1_App_Seats.csv'
candidate_path = '/home/it/Desktop/RUN/NEUT_Official/NEUT_R1/OfficialData/R1_App_CandidateProfile.csv'
# choice_path = '/home/it/Desktop/RUN/NEUT_R1/Data/App_Choice.csv'
choice_path = '/home/it/Desktop/RUN/NEUT_Official/NEUT_R1/OfficialData/R1_App_Choice.csv'
# seats_path = '/home/it/Desktop/RUN/NEUT_R1/Data/App_Seat.csv'
# category_path='/home/it/Desktop/RUN/NEUT_R1/Data/MD_Category.csv'
# sub_category_path='/home/it/Desktop/RUN/NEUT_R1/Data/MD_Category.csv'

eligible_candidates_path = '/home/it/Desktop/RUN/NEUT_Official/NEUT_R1/OfficialData/R1_App_CandidateProfile.csv' # Switched fpr eligibale candidates

In [20]:


candidate = pd.read_csv(candidate_path)
reordered_columns = ['rollno', 'boardId','stateId', 'domicileId', 'genderId','categoryId', 'subCategoryList','categoryCertificateNo']
candidate = candidate[reordered_columns]

# category = pd.read_csv(category_path)
# sub_category = pd.read_csv(sub_category_path)

choice = pd.read_csv(choice_path)
choice = choice.rename(columns={'rollNo': 'rollno', 'programId':'program_name','instituteId':'program_code','optNo':'ChoiceNumber'})
reordered_columns = ['rollno', 'program_name','program_code','ChoiceNumber']
choice = choice[reordered_columns]

eligible_candidates = pd.read_csv(eligible_candidates_path)
eligible_candidates = eligible_candidates.rename(columns={'rollNo': 'rollno'})

# eligible_candidates = eligible_candidates[eligible_candidates['activityId'] == 26]  #since no need to filter

eligible_candidates = eligible_candidates['rollno']
candidate_filtered = pd.merge(eligible_candidates, candidate, on="rollno")
candidate_choice_merge = pd.merge(choice, candidate_filtered, on="rollno")
candidate_choice_merge.to_csv('candidate_choice_merge.csv', index=False)
# seat = pd.read_csv(seats_path)
candidate_choice_merge


,rollno,program_name,program_code,ChoiceNumber,boardId,stateId,domicileId,genderId,categoryId,subCategoryList,categoryCertificateNo
0,240310000054,CHT,34,1,104022421,32.0,32,1,11,"PH:No,PR:Yes",NaN
1,240310000054,ETC,200,2,104022421,32.0,32,1,11,"PH:No,PR:Yes",NaN
2,240310000054,ESE,200,3,104022421,32.0,32,1,11,"PH:No,PR:Yes",NaN
3,240310000639,CSS,237,1,104022421,4.0,4,1,11,"PH:No,PR:Yes",NaN
4,240310000639,ECE,237,2,104022421,4.0,4,1,11,"PH:No,PR:Yes",NaN
...,...,...,...,...,...,...,...,...,...,...,...
15737,240410000259,PHA,213,2,104032421,NaN,8,2,11,"PH:No,25:Yes,26:No,27:No,28:No",NaN
15738,240410000259,PHA,33,3,104032421,NaN,8,2,11,"PH:No,25:Yes,26:No,27:No,28:No",NaN
15739,240410000265,PHA,72,1,104032421,NaN,3,2,8,PH:No,NaN
15740,240410000275,PHA,205,1,104032421,NaN,23,1,26,PH:No,NaN


In [21]:
candidate_choice_merge['subCategoryList'].unique

<bound method Series.unique of 0                          PH:No,PR:Yes
1                          PH:No,PR:Yes
2                          PH:No,PR:Yes
3                          PH:No,PR:Yes
4                          PH:No,PR:Yes
                      ...              
15737    PH:No,25:Yes,26:No,27:No,28:No
15738    PH:No,25:Yes,26:No,27:No,28:No
15739                             PH:No
15740                             PH:No
15741                             PH:No
Name: subCategoryList, Length: 15742, dtype: object>

In [22]:
# institutes={}
# for index, row in seat.iterrows():
#     if row[' Quota']=='OS' or row[' Quota']=='HS':
#         if not row['InstCode'] in institutes.keys():
#             institutes[row['InstCode']]=[row['Stcode1'],row['Stcode2'],row['Stcode3'],row['Stcode4'],row['Stcode5'],row['Stcode6'],row['Stcode7'],row['Stcode8'],row['Stcode9'],row['Stcode10']]

In [23]:
# def determine_virtual_candidate_category(row):
#     if row['domicileId'] == 'Arunachal Pradesh' and row['categoryId'] == "Category_I":
#         return['Category_I', 'Category_I_PWD']
#     elif row['domicileId'] == 'Arunachal Pradesh' and row['categoryId'] == "Category_II":
#         return['Category_II', 'Category_II_PWD']

Nirst_dom = [3,4,22,23,24,25,30,32]

def determine_virtual_domicile(row):
    curr_dom=str(row['domicileId'])
    if len(curr_dom)==1:
        curr_dom=f"0{curr_dom}"
    if row['program_code'] ==  237 :
        if row['domicileId'] in Nirst_dom:
            return ['NE', curr_dom]
    return [curr_dom]

def determine_virtual_categories(row):
    # if row['categoryId'] == 'BC':
    #     return ['OP', 'BC']
    # elif row['categoryId'] == 'OP':
    #     return ['OP']
    # elif row['categoryId'] == 'EW':
    #     return ['OP', 'EW']
    # elif row['categoryId'] == 'SC':
    #     return ['OP', 'SC']
    # elif row['categoryId'] == 'ST':
    #     return ['OP', 'ST']
    # else:
    #     return []
    if row['Dom_V'] == 'NE':
        return [11]
    if row['domicileId'] == 3:  # Arunachal Pradesh 
        if row['categoryId'] == 1 or row['categoryId']==8:
            return [1]
        elif row['categoryId'] == 2 or (row['categoryId']>=32 and row['categoryId']<=34):
            return [2]
        # return row['categoryId']

    elif row['domicileId'] == 4:  # Assam 
        if row['categoryId'] == 11:  # 11 is for UR
            return [11]
        elif row['categoryId'] == 20:  # 20 is for SC
            return [11, 20]
        elif row['categoryId'] == 27:  # 27 is for ST P
            return [11, 27]
        elif row['categoryId'] == 25:  # 25 is for SC HILLS
            return [11, 25]
        elif row['categoryId'] == 39:  # 39 is for OBC
            return [11, 39]

    elif row['domicileId'] == 22:  # Manipur
        if row['categoryId'] == 11:  # 11 is for UR
            return [11]
        elif row['categoryId'] == 20:  # 20 is for SC
            return [11, 20]
        elif row['categoryId'] == 22:  # 22 is for ST
            return [11, 22]
        elif row['categoryId'] == 30:  # 30 is for OBC (Meitei)
            return [11, 30]
        elif row['categoryId'] == 18:  # 18 is for OBC (Meitei-Pangal)
            return [11, 18]
        elif row['categoryId'] == 31:  # 31 is for OBC (Telli, Others)
            return [11, 31]

    elif row['domicileId'] == 23:  # Meghalaya
        if row['categoryId'] == 11:  # 11 is for UR
            return [11]
        elif row['categoryId'] == 26:  # 26 is for ST Khasi and Jaintia
            return [11, 26]
        elif row['categoryId'] == 24:  # 24 is for ST Garo
            return [11, 24]
        elif row['categoryId'] == 29:  # 29 is for Other ST and SC
            return [11, 29]

    elif row['domicileId'] == 24:  # Mizoram
        # if row['categoryId'] == 42:  # 42 is for Open
        #     return [42]
        # elif row['categoryId'] == 1:  # 1 is for CATEGORY-1
        #     return [1]
        # elif row['categoryId'] == 2:  # 2 is for CATEGORY-2
        #     return [2]
        # elif row['categoryId'] == 3:  # 3 is for CATEGORY-3
        #     return [3]
        # elif row['categoryId'] == 6:  # 6 is for CATEGORY-4
        #     return [6]
        return row['categoryId']

    elif row['domicileId'] == 25:  # Nagaland
        if row['categoryId'] == 40:  # 40 is for CATEGORY-1 (ST)
            return [40]
        elif row['categoryId'] == 41:  # 41 is for CATEGORY-2 (ST-BT)
            return [40, 41]
        elif row['categoryId'] == 37:  # 37 is for CATEGORY-3 SC
            return [37]
        elif row['categoryId'] == 38:  # 38 is for CATEGORY-3 ST
            return [38]
        elif row['categoryId'] == 3:  # 3 is for CATEGORY 3
            return [3]
        # return row['categoryId']

    elif row['domicileId'] == 30:  # Sikkim
        if row['categoryId'] == 11:  # 11 is for UR
            return [11]
        elif row['categoryId'] == 9:  # 9 is for Bhutia and Lepcha Non BPL
            return [11, 9]
        elif row['categoryId'] == 10:  # 10 is for Bhutia and Lepcha BPL
            return [11, 10]
        elif row['categoryId'] == 19:  # 19 is for Primitive Tribe
            return [11, 19]
        elif row['categoryId'] == 14:  # 14 is for OBC Central Non BPL
            return [11, 14]
        elif row['categoryId'] == 15:  # 15 is for OBC Central BPL
            return [11, 15]
        elif row['categoryId'] == 16:  # 16 is for OBC State Non BPL
            return [11, 16]
        elif row['categoryId'] == 17:  # 17 is for OBC State BPL
            return [11, 17]
        elif row['categoryId'] == 20:  # 20 is for SC
            return [11, 20]
        elif row['categoryId'] == 23:  # 23 is for ST Non BPL
            return [11, 23]
        elif row['categoryId'] == 28:  # 28 is for ST BPL
            return [11, 28]
        elif row['categoryId'] == 21:  # 21 is for Others
            return [21]
        # return row['categoryId'] For 2024

    elif row['domicileId'] == 32:  # Tripura
        if row['categoryId'] == 11:  # 11 is for UR
            return [11]
        elif row['categoryId'] == 20:  # 20 is for SC
            return [11, 20]
        elif row['categoryId'] == 22:  # 22 is for ST
            return [11, 22]

    #Not needed for now
    # elif row['domicileId'] == 1:  # Andaman and Nicobar
    #     # if row['categoryId'] == 1:  # 1 is for CATEGORY-1
    #     #     return [1]
    #     # elif row['categoryId'] == 2:  # 2 is for CATEGORY-1
    #     #     return [2]
    #     # elif row['categoryId'] == 4:  # 4 is for CATEGORY-3A
    #     #     return [4]
    #     # elif row['categoryId'] == 5:  # 5 is for CATEGORY-3B
    #     #     return [5]
    #     # elif row['categoryId'] == 6:  # 6 is for CATEGORY-4
    #     #     return [6]
    #     # elif row['categoryId'] == 7:  # 7 is for CATEGORY-5
    #     #     return [7]
    #     return row['categoryId']

    elif row['domicileId'] == 8:  # Dadra and Nagar Haveli
        if row['categoryId'] == 11:  # 11 is for UR
            return [11]
        elif row['categoryId'] == 13:  # 13 is for OBC
            return [11, 13]
        elif row['categoryId'] == 20:  # 20 is for SC
            return [11, 20]
        elif row['categoryId'] == 22:  # 22 is for ST
            return [11, 22]

    elif row['domicileId'] == 9:  # Daman and Diu
        if row['categoryId'] == 11:  # 11 is for UR
            return [11]
        elif row['categoryId'] == 43:  # 43 is for UR-EWS
            return [11, 43]
        elif row['categoryId'] == 20:  # 20 is for SC
            return [11, 20]
        elif row['categoryId'] == 22:  # 22 is for ST
            return [11, 22]
        elif row['categoryId'] == 13:  # 13 is for OBC
            return [11, 13]

    elif row['domicileId'] == 37:  # Ladakh
        #Note 11 is there in 2024 but 42 in 2023
        if row['categoryId'] == 42:  # 42 is for Open
            return [42]
        elif row['categoryId'] == 22:  # 22 is for ST
            return [42, 22]
        elif row['categoryId'] == 20:  # 20 is for SC
            return [42, 20]
        elif row['categoryId'] == 44:  # 44 is for ALC
            return [42, 44]
        elif row['categoryId'] == 43:  # 43 is for UR-EWS
            return [42, 43]

    elif row['domicileId'] == 19:  # Lakshadweep
        if row['categoryId'] == 11:  # 11 is for UR
            return [11]
        elif row['categoryId'] == 1:  # 1 is for CATEGORY-1
            return [11, 1]
        elif row['categoryId'] == 2:  # 2 is for CATEGORY-2 (Vacant seats of CATEGORY-1)
            return [2]
        elif row['categoryId'] == 3:  # 3 is for CATEGORY-3 (Vacant seats of CATEGORY-1 and CATEGORY-2)
            return [3]

    return []  # Default return value if no conditions are met

    
def determine_virtual_sub_categories(row):
    if "PH:No" in row['subCategoryList'] :
        return ['NO']
    elif "PH:Yes" in row['subCategoryList'] :
        return ['NO', 'PH']
    else:
        return []

def determine_virtual_gender(row):
    if row['Gender'] == 'F':
        return ['F', 'B']
    elif row['Gender'] == 'B':
        return ['B']
    else:
        return []


# def process_df(df):
    # df = df[
    #     # (df['RStatus'] == 'RP') & 
    #     # (df['Withdraw'] == 'N') & 
    #     # (df['Validity'] != 'N') &    
    #     (
    #         (df['Decision'] == 'FL') | 
    #         ((df['Decision'] == 'FR') & (df['ChoiceNumber'] >= df['AllotedOpt'])) | 
    #         ((df['Decision'] == 'SL') & 
    #         ((df['ChoiceNumber'] >= df['AllotedOpt']) | ((df['ChoiceNumber'] < df['AllotedOpt']) & df['Instcd'].notna() & (df['Instcd'].fillna(-1).astype(int) == df['InstCode']))))
    #     )
    # ]
    # df['Dom_V'] = df.apply(determine_virtual_domicile, axis=1)
    # df = df.explode('Dom_V')
    # df['Cat_V'] = df.apply(determine_virtual_categories, axis=1)
    # df = df.explode('Cat_V')
    # df['SubCat_V'] = df.apply(determine_virtual_sub_categories, axis=1)
    # df = df.explode('SubCat_V')
    # df['State_quota_V'] = df.apply(determine_virtual_state_quota, axis=1)
    # df = df.explode('State_quota_V')
    # df['Gender_V'] = df.apply(determine_virtual_gender, axis=1)
    # df = df.explode('Gender_V')
    # df['State_quota_V'] = df.apply(determine_virtual_state_quota, axis=1)
    # df = df.explode('State_quota_V')
    # return df
    
def explode_dom(df):
    df['Dom_V'] = df.apply(determine_virtual_domicile, axis=1)
    df = df.explode('Dom_V')
    return df

def explode_cat(df):
    df['Cat_V'] = df.apply(determine_virtual_categories, axis=1)
    df = df.explode('Cat_V')
    return df

def explode_subcat(df):
    df['SubCat_V'] = df.apply(determine_virtual_sub_categories, axis=1)
    df = df.explode('SubCat_V')
    return df

start_time_loading_merging = time.time()

merged_ch_df = pd.read_csv(r'candidate_choice_merge.csv')

# Process the DataFrame
merged_df1 = explode_dom(merged_ch_df)
merged_df2 = explode_cat(merged_df1)
merged_df = explode_subcat(merged_df2)


end_time_loading_merging = time.time()
print(f"Time taken for loading tables and merging and running virtulization: {end_time_loading_merging - start_time_loading_merging:.2f} seconds")
# Correct the first line
merged_df.loc[(merged_df['Dom_V'] == 'NE') & (merged_df['SubCat_V'] == 'NO'), 'SubCat_V'] = 'PR'

# Correct the second line
merged_df = merged_df[~((merged_df['SubCat_V'] == 'PR') & ~(merged_df['subCategoryList'].str.contains('PR:Yes')))]

print(f"Now Sorting:")
merged_df.sort_values(by=['rollno', 'ChoiceNumber'], inplace=True)
end_time_sorting = time.time()

print(f"Time taken for Sorting: {end_time_sorting - end_time_loading_merging:.2f} seconds")

print(f"Now creating Virtual count")
merged_df['ChoiceNumber_V'] = merged_df.groupby(['rollno']).cumcount() + 1
end_time_V = time.time()
print(f"Time taken for creating Virtual count: {end_time_V- end_time_sorting:.2f} seconds")


# # Ensure Adv_DS_V column is empty initially
# merged_df['Adv_DS_V'] = None

# # Identify rollnos with Adv_DS == 1.0
# adv_ds_rollnos = merged_df[merged_df['Adv_DS'] == 1.0]['rollno'].unique()

# ds_df = merged_df[merged_df['rollno'].isin(adv_ds_rollnos)]
# normal_df = merged_df[~merged_df['rollno'].isin(adv_ds_rollnos)]

# # Create new rows for DS rollnos at the end of each choice
# new_rows = []
# for rollno in adv_ds_rollnos:
#     unique_choices = ds_df[ds_df['rollno'] == rollno]['ChoiceNumber'].unique()
#     for choice in unique_choices:
#         choice_rows = ds_df[(ds_df['rollno'] == rollno) & (ds_df['ChoiceNumber'] == choice)]
#         if choice_rows.iloc[0]['InstCode'] < 200:  # Only add DS rows for IITs
#             last_virtual_choice_no = choice_rows['ChoiceNumber_V'].max() + 1
#             last_row = choice_rows.iloc[-1]
#             new_row = last_row.copy()
#             new_row['ChoiceNumber_V'] = last_virtual_choice_no
#             new_row['Adv_DS_V'] = 1
#             new_rows.append(new_row)

# new_rows_df = pd.DataFrame(new_rows)

# # Append new rows to the DS DataFrame
# ds_final_df = pd.concat([ds_df, new_rows_df], ignore_index=True)

# # Recalculate VirtualChoiceNumber to account for new DS rows
# ds_final_df.sort_values(by=['rollno', 'ChoiceNumber'], inplace=True)
# ds_final_df['ChoiceNumber_V'] = ds_final_df.groupby(['rollno']).cumcount() + 1

# Select columns to reorder them as needed
reordered_columns = ['rollno',  'program_name', 'program_code', 'boardId',
       'stateId', 'genderId',  
       'categoryCertificateNo','domicileId','Dom_V', 'categoryId', 'Cat_V','subCategoryList', 'SubCat_V','ChoiceNumber','ChoiceNumber_V']

normal_df = merged_df[reordered_columns]
# normal_df.loc[normal_df['Dom_V'] == 'NE', 'Cat_V'] = 11

print(f"Now creating csv file")
normal_df.to_csv('Raw_Virtualized_Choices.csv', index=False)
# ds_final_df.to_csv('DS_Virtualized_Choices_R7.csv', index=False)


print(f"Time taken for creating csv file: {time.time() - end_time_V:.2f} seconds")

Time taken for loading tables and merging and running virtulization: 0.54 seconds
Now Sorting:
Time taken for Sorting: 0.01 seconds
Now creating Virtual count
Time taken for creating Virtual count: 0.00 seconds
Now creating csv file
Time taken for creating csv file: 0.08 seconds


In [24]:
merged_df2

,rollno,program_name,program_code,ChoiceNumber,boardId,stateId,domicileId,genderId,categoryId,subCategoryList,categoryCertificateNo,Dom_V,Cat_V,SubCat_V
0,240310000054,CHT,34,1,104022421,32.0,32,1,11,"PH:No,PR:Yes",NaN,32,11,[NO]
1,240310000054,ETC,200,2,104022421,32.0,32,1,11,"PH:No,PR:Yes",NaN,32,11,[NO]
2,240310000054,ESE,200,3,104022421,32.0,32,1,11,"PH:No,PR:Yes",NaN,32,11,[NO]
3,240310000639,CSS,237,1,104022421,4.0,4,1,11,"PH:No,PR:Yes",NaN,NE,11,[NO]
3,240310000639,CSS,237,1,104022421,4.0,4,1,11,"PH:No,PR:Yes",NaN,04,11,[NO]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15739,240410000265,PHA,72,1,104032421,NaN,3,2,8,PH:No,NaN,03,1,[NO]
15740,240410000275,PHA,205,1,104032421,NaN,23,1,26,PH:No,NaN,23,11,[NO]
15740,240410000275,PHA,205,1,104032421,NaN,23,1,26,PH:No,NaN,23,26,[NO]
15741,240410000275,PHA,204,2,104032421,NaN,23,1,26,PH:No,NaN,23,11,[NO]


In [25]:
merged_df.columns

Index(['rollno', 'program_name', 'program_code', 'ChoiceNumber', 'boardId',
       'stateId', 'domicileId', 'genderId', 'categoryId', 'subCategoryList',
       'categoryCertificateNo', 'Dom_V', 'Cat_V', 'SubCat_V',
       'ChoiceNumber_V'],
      dtype='object')

In [26]:
end_full = time.time()
total_time= end_full -start_full
print(f"Total time taken  - {total_time}")
print(f" In minutes - {total_time/60}")

Total time taken  - 0.8722250461578369
 In minutes - 0.014537084102630615


In [27]:
class Program:
    def __init__(self, quotaId,instituteId,programId,categoryId,subcategoryId,tSeat):
        self.quota = quotaId
        self.institute = instituteId
        self.program = programId
        self.category = categoryId
        self.subcategory = subcategoryId
        self.capacity = tSeat


# boardId,roundNo,streamId,groupId,seatType,quotaId,instituteId,programId,categoryId,subcategoryId,genderId,rankTypeId,tSeat,pSeat,aSeat,rSeat,vSeat
# Function to load candidates from the CSV file
def load_programs(filename):
    df = pd.read_csv(filename)
    df.loc[df['subcategoryId'] == 'No', 'subcategoryId'] = 'NO'
    return {(row['instituteId'], row['programId'], row['quotaId'], row['categoryId'], row['subcategoryId']): Program(row['quotaId'],
        row['instituteId'],
        row['programId'],
        row['categoryId'],
        row['subcategoryId'],
        row['tSeat']) for index, row in df.iterrows()}

# Load candidates from the CandidateProfile.csv file
programs = load_programs(seats_path)
all_progs = set()
for key in programs.keys():
    all_progs.add(key)
filt_progs = set()
for key in programs.keys():
    if programs[key].capacity==0:
        filt_progs.add(key)



In [28]:
for key in programs.keys():
    if programs[key].quota=='NE':
        print(f"{key} has {programs[key].capacity} seats")

(237, 'AGE', 'NE', 11, 'PH') has 0 seats
(237, 'AGE', 'NE', 11, 'PR') has 1 seats
(237, 'CSS', 'NE', 11, 'PR') has 3 seats
(237, 'CVE', 'NE', 11, 'PH') has 0 seats
(237, 'CVE', 'NE', 11, 'PR') has 1 seats
(237, 'ECE', 'NE', 11, 'PH') has 1 seats
(237, 'ECE', 'NE', 11, 'PR') has 0 seats
(237, 'ELE', 'NE', 11, 'PH') has 1 seats
(237, 'ELE', 'NE', 11, 'PR') has 0 seats
(237, 'MEE', 'NE', 11, 'PH') has 1 seats
(237, 'MEE', 'NE', 11, 'PR') has 0 seats


In [29]:
filt_progs

{(183, 'ARC', '30', 28, 'NO'),
 (138, 'TET', '04', 11, 'NO'),
 (254, 'ARC', '01', 35, 'NO'),
 (15, 'EEE', '01', 1, 'PH'),
 (115, 'ESE', '24', 45, 'PH'),
 (58, 'CET', '22', 13, 'NO'),
 (90, 'CTE', '22', 22, 'NO'),
 (140, 'ETE', '24', 3, 'PH'),
 (91, 'CTT', '22', 18, 'NO'),
 (251, 'AEI', '24', 6, 'PH'),
 (158, 'ELE', '25', 41, 'PH'),
 (54, 'CVE', '03', 1, 'PH'),
 (35, 'ETC', '23', 26, 'PH'),
 (5, 'CHE', '22', 31, 'NO'),
 (141, 'CVE', '24', 6, 'PH'),
 (200, 'MEE', '23', 29, 'NO'),
 (59, 'CPE', '08', 20, 'NO'),
 (21, 'ICE', '01', 4, 'NO'),
 (20, 'EEE', '01', 6, 'PH'),
 (74, 'CPE', '24', 6, 'NO'),
 (46, 'CSS', '01', 35, 'NO'),
 (170, 'MEE', '30', 20, 'NO'),
 (162, 'ETE', '25', 41, 'PH'),
 (107, 'ARC', '22', 30, 'NO'),
 (150, 'ETC', '24', 6, 'NO'),
 (194, 'MEE', '30', 11, 'NO'),
 (128, 'CPE', '24', 45, 'PH'),
 (105, 'ECE', '22', 18, 'NO'),
 (251, 'CTE', '24', 45, 'PH'),
 (100, 'ETC', '22', 30, 'NO'),
 (46, 'CHT', '30', 14, 'NO'),
 (54, 'CTE', '22', 20, 'NO'),
 (50, 'ELE', '23', 29, 'NO'),
 (

In [30]:
all_progs

{(183, 'ARC', '30', 28, 'NO'),
 (138, 'TET', '04', 11, 'NO'),
 (254, 'ARC', '01', 35, 'NO'),
 (15, 'EEE', '01', 1, 'PH'),
 (115, 'ESE', '24', 45, 'PH'),
 (58, 'CET', '22', 13, 'NO'),
 (90, 'CTE', '22', 22, 'NO'),
 (9, 'CVE', '24', 45, 'NO'),
 (91, 'CTT', '22', 18, 'NO'),
 (61, 'ELE', '03', 2, 'NO'),
 (140, 'ETE', '24', 3, 'PH'),
 (99, 'ETC', '24', 45, 'NO'),
 (54, 'CVE', '03', 1, 'PH'),
 (35, 'ETC', '23', 26, 'PH'),
 (5, 'CHE', '22', 31, 'NO'),
 (141, 'CVE', '24', 6, 'PH'),
 (251, 'AEI', '24', 6, 'PH'),
 (59, 'CTE', '09', 11, 'NO'),
 (200, 'MEE', '23', 29, 'NO'),
 (59, 'CPE', '08', 20, 'NO'),
 (21, 'ICE', '01', 4, 'NO'),
 (54, 'ECE', '22', 22, 'NO'),
 (20, 'EEE', '01', 6, 'PH'),
 (74, 'CPE', '24', 6, 'NO'),
 (46, 'CSS', '01', 35, 'NO'),
 (170, 'MEE', '30', 20, 'NO'),
 (162, 'ETE', '25', 41, 'PH'),
 (107, 'ARC', '22', 30, 'NO'),
 (150, 'ETC', '24', 6, 'NO'),
 (194, 'MEE', '30', 11, 'NO'),
 (128, 'CPE', '24', 45, 'PH'),
 (20, 'EEE', '22', 11, 'NO'),
 (105, 'ECE', '22', 18, 'NO'),
 (251, 

In [31]:
last_df=normal_df
# for rollno,program_name,program_code,boardId,stateId,genderId,categoryCertificateNo,domicileId,Dom_V,categoryId,Cat_V,subCategoryList,SubCat_V,ChoiceNumber,ChoiceNumber_V in last_df:
#     if (program_code, program_name, Dom_V, Cat_V, SubCat_V) in filt_progs:
#         del this row plij from last df 

# return the corrected df into a csv

In [32]:
def filter_rows1(df, all_progs):
    condition = df.apply(lambda row: (row['program_code'], row['program_name'], row['Dom_V'], row['Cat_V'], row['SubCat_V']) in all_progs, axis=1)
    return df[condition]
def filter_rows2(df, filt_progs):
    condition = df.apply(lambda row: (row['program_code'], row['program_name'], row['Dom_V'], row['Cat_V'], row['SubCat_V']) in filt_progs, axis=1)
    return df[~condition]

last_df_filtered = filter_rows1(last_df, all_progs)
last_df_filtered1 = filter_rows2(last_df_filtered, filt_progs)

print(last_df_filtered1)

print(f"Now Sorting:")
last_df_filtered.sort_values(by=['rollno', 'ChoiceNumber'], inplace=True)
end_time_sorting = time.time()

print(f"Time taken for Sorting: {end_time_sorting - end_time_loading_merging:.2f} seconds")

# print(f"Now Sorting:")
# last_df_filtered01 = last_df_filtered1[(last_df_filtered1['Dom_V'] != '25') | (last_df_filtered1['Cat_V'] != 41)]
# last_df_filtered01.sort_values(by=['rollno', 'ChoiceNumber'], inplace=True)
# end_time_sorting = time.time()
# print(f"Time taken for Sorting: {end_time_sorting - end_time_loading_merging:.2f} seconds")
# print(f"Now creating Virtual count")
# last_df_filtered1['ChoiceNumber_V'] = last_df_filtered1.groupby(['rollno']).cumcount() + 1
# end_time_V = time.time()
# print(f"Time taken for creating Virtual count: {end_time_V- end_time_sorting:.2f} seconds")

# print("Now Sorting:")
# last_df_filtered02 = last_df_filtered1[(last_df_filtered1['Dom_V'] == '25') & (last_df_filtered1['Cat_V'] == 41)]
# last_df_filtered02.sort_values(by=['rollno', 'ChoiceNumber'], inplace=True)
# end_time_sorting = time.time()
# print(f"Time taken for Sorting: {end_time_sorting - end_time_loading_merging:.2f} seconds")
# print(f"Now creating Virtual count")
# last_df_filtered02['ChoiceNumber_V'] = last_df_filtered02.groupby(['rollno']).cumcount() + 1
# end_time_V = time.time()
# print(f"Time taken for creating Virtual count: {end_time_V- end_time_sorting:.2f} seconds")

# last_df_filtered1 = pd.concat([last_df_filtered01, last_df_filtered02])

print(f"Now creating Virtual count")
# last_df_filtered1.sort_values(by=['rollno', 'ChoiceNumber_V'], inplace=True)
last_df_filtered['ChoiceNumber_V'] = last_df_filtered.groupby(['rollno']).cumcount() + 1
end_time_V = time.time()
print(f"Time taken for creating Virtual count: {end_time_V- end_time_sorting:.2f} seconds")

last_df_filtered.to_csv('VirtualizedChoices1.csv', index=False)


             rollno program_name  program_code    boardId  stateId  genderId  \
0      240310000054          CHT            34  104022421     32.0         1   
1      240310000054          ETC           200  104022421     32.0         1   
2      240310000054          ESE           200  104022421     32.0         1   
3      240310000639          CSS           237  104022421      4.0         1   
3      240310000639          CSS           237  104022421      4.0         1   
...             ...          ...           ...        ...      ...       ...   
15735  240410000258          PHA           213  104032421      NaN         2   
15737  240410000259          PHA           213  104032421      NaN         2   
15738  240410000259          PHA            33  104032421      NaN         2   
15739  240410000265          PHA            72  104032421      NaN         2   
15741  240410000275          PHA           204  104032421      NaN         1   

       categoryCertificateNo  domicileI

/tmp/ipykernel_98419/3540977262.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_df_filtered.sort_values(by=['rollno', 'ChoiceNumber'], inplace=True)
/tmp/ipykernel_98419/3540977262.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_df_filtered['ChoiceNumber_V'] = last_df_filtered.groupby(['rollno']).cumcount() + 1


In [33]:
def filter_rows1(df, all_progs):
    condition = df.apply(lambda row: (row['program_code'], row['program_name'], row['Dom_V'], row['Cat_V'], row['SubCat_V']) in all_progs, axis=1)
    return df[condition]
def filter_rows2(df, filt_progs):
    condition = df.apply(lambda row: (row['program_code'], row['program_name'], row['Dom_V'], row['Cat_V'], row['SubCat_V']) in filt_progs, axis=1)
    return df[~condition]

last_df_filtered = filter_rows1(last_df, all_progs)
last_df_filtered1 = filter_rows2(last_df_filtered, filt_progs)

print(last_df_filtered1)

print(f"Now Sorting:")
last_df_filtered.sort_values(by=['rollno', 'ChoiceNumber'], inplace=True)
end_time_sorting = time.time()

print(f"Time taken for Sorting: {end_time_sorting - end_time_loading_merging:.2f} seconds")

# print(f"Now Sorting:")
# last_df_filtered01 = last_df_filtered1[(last_df_filtered1['Dom_V'] != '25') | (last_df_filtered1['Cat_V'] != 41)]
# last_df_filtered01.sort_values(by=['rollno', 'ChoiceNumber'], inplace=True)
# end_time_sorting = time.time()
# print(f"Time taken for Sorting: {end_time_sorting - end_time_loading_merging:.2f} seconds")
# print(f"Now creating Virtual count")
# last_df_filtered1['ChoiceNumber_V'] = last_df_filtered1.groupby(['rollno']).cumcount() + 1
# end_time_V = time.time()
# print(f"Time taken for creating Virtual count: {end_time_V- end_time_sorting:.2f} seconds")

# print("Now Sorting:")
# last_df_filtered02 = last_df_filtered1[(last_df_filtered1['Dom_V'] == '25') & (last_df_filtered1['Cat_V'] == 41)]
# last_df_filtered02.sort_values(by=['rollno', 'ChoiceNumber'], inplace=True)
# end_time_sorting = time.time()
# print(f"Time taken for Sorting: {end_time_sorting - end_time_loading_merging:.2f} seconds")
# print(f"Now creating Virtual count")
# last_df_filtered02['ChoiceNumber_V'] = last_df_filtered02.groupby(['rollno']).cumcount() + 1
# end_time_V = time.time()
# print(f"Time taken for creating Virtual count: {end_time_V- end_time_sorting:.2f} seconds")

# last_df_filtered1 = pd.concat([last_df_filtered01, last_df_filtered02])

print(f"Now creating Virtual count")
# last_df_filtered1.sort_values(by=['rollno', 'ChoiceNumber_V'], inplace=True)
last_df_filtered['ChoiceNumber_V'] = last_df_filtered.groupby(['rollno']).cumcount() + 1
end_time_V = time.time()
print(f"Time taken for creating Virtual count: {end_time_V- end_time_sorting:.2f} seconds")

last_df_filtered.to_csv('VirtualizedChoices1.csv', index=False)


             rollno program_name  program_code    boardId  stateId  genderId  \
0      240310000054          CHT            34  104022421     32.0         1   
1      240310000054          ETC           200  104022421     32.0         1   
2      240310000054          ESE           200  104022421     32.0         1   
3      240310000639          CSS           237  104022421      4.0         1   
3      240310000639          CSS           237  104022421      4.0         1   
...             ...          ...           ...        ...      ...       ...   
15735  240410000258          PHA           213  104032421      NaN         2   
15737  240410000259          PHA           213  104032421      NaN         2   
15738  240410000259          PHA            33  104032421      NaN         2   
15739  240410000265          PHA            72  104032421      NaN         2   
15741  240410000275          PHA           204  104032421      NaN         1   

       categoryCertificateNo  domicileI

/tmp/ipykernel_98419/3540977262.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_df_filtered.sort_values(by=['rollno', 'ChoiceNumber'], inplace=True)
/tmp/ipykernel_98419/3540977262.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_df_filtered['ChoiceNumber_V'] = last_df_filtered.groupby(['rollno']).cumcount() + 1


In [34]:
# def filter_rows1(df, all_progs):
#     condition = df.apply(lambda row: (row['program_code'], row['program_name'], row['Dom_V'], row['Cat_V'], row['SubCat_V']) in all_progs, axis=1)
#     return df[condition]
# def filter_rows2(df, filt_progs):
#     condition = df.apply(lambda row: (row['program_code'], row['program_name'], row['Dom_V'], row['Cat_V'], row['SubCat_V']) in filt_progs, axis=1)
#     return df[~condition]

# last_df_filtered = filter_rows1(last_df, all_progs)
# last_df_filtered1 = filter_rows2(last_df_filtered, filt_progs)

# print(last_df_filtered1)

# print(f"Now Sorting:")
# last_df_filtered1.sort_values(by=['rollno', 'ChoiceNumber'], inplace=True)
# end_time_sorting = time.time()

# print(f"Time taken for Sorting: {end_time_sorting - end_time_loading_merging:.2f} seconds")

# # print(f"Now Sorting:")
# # last_df_filtered01 = last_df_filtered1[(last_df_filtered1['Dom_V'] != '25') | (last_df_filtered1['Cat_V'] != 41)]
# # last_df_filtered01.sort_values(by=['rollno', 'ChoiceNumber'], inplace=True)
# # end_time_sorting = time.time()
# # print(f"Time taken for Sorting: {end_time_sorting - end_time_loading_merging:.2f} seconds")
# # print(f"Now creating Virtual count")
# # last_df_filtered1['ChoiceNumber_V'] = last_df_filtered1.groupby(['rollno']).cumcount() + 1
# # end_time_V = time.time()
# # print(f"Time taken for creating Virtual count: {end_time_V- end_time_sorting:.2f} seconds")

# # print("Now Sorting:")
# # last_df_filtered02 = last_df_filtered1[(last_df_filtered1['Dom_V'] == '25') & (last_df_filtered1['Cat_V'] == 41)]
# # last_df_filtered02.sort_values(by=['rollno', 'ChoiceNumber'], inplace=True)
# # end_time_sorting = time.time()
# # print(f"Time taken for Sorting: {end_time_sorting - end_time_loading_merging:.2f} seconds")
# # print(f"Now creating Virtual count")
# # last_df_filtered02['ChoiceNumber_V'] = last_df_filtered02.groupby(['rollno']).cumcount() + 1
# # end_time_V = time.time()
# # print(f"Time taken for creating Virtual count: {end_time_V- end_time_sorting:.2f} seconds")

# # last_df_filtered1 = pd.concat([last_df_filtered01, last_df_filtered02])

# print(f"Now creating Virtual count")
# # last_df_filtered1.sort_values(by=['rollno', 'ChoiceNumber_V'], inplace=True)
# last_df_filtered1['ChoiceNumber_V'] = last_df_filtered1.groupby(['rollno']).cumcount() + 1
# end_time_V = time.time()
# print(f"Time taken for creating Virtual count: {end_time_V- end_time_sorting:.2f} seconds")

# last_df_filtered1.to_csv('VirtualizedChoices.csv', index=False)


In [35]:
import pandas as pd

# File path
inputfile = '/home/it/Desktop/RUN/NEUT_Official/NEUT_R1/Codes/CSAB_NEUT_R1_Allotment_Final.csv'
df = pd.read_csv(inputfile)
dom = pd.read_csv('/home/it/Desktop/RUN/NEUT_Official/NEUT_R1/OfficialData/R1_MD_Domicile.csv')
inst = pd.read_csv('/home/it/Desktop/RUN/NEUT_Official/NEUT_R1/OfficialData/R1_MD_Institute.csv')
prog = pd.read_csv('/home/it/Desktop/RUN/NEUT_Official/NEUT_R1/OfficialData/R1_MD_Program.csv')
cat = pd.read_csv('/home/it/Desktop/RUN/NEUT_Official/NEUT_R1/OfficialData/R1_MD_Category.csv')
df['Application ID']=df['Roll No']

# dom['Quota']=dom['id']
# merged = pd.merge(df,dom,on='Quota')
# merged['Alloted Quota']=merged['description']
# cols= [ 'Application ID','Alloted Quota','Program Code','Branch Code']
# merged = merged[cols]


# inst['Program Code']=inst['id']
# merged = pd.merge(merged,inst,on='Program Code')
# merged['Alloted Institute']=merged['description']
# cols= [ 'Application ID','Alloted Quota','Alloted Institute','Branch Code']
# merged = merged[cols]


# prog['Branch Code']=prog['id']
# merged = pd.merge(merged,prog,on='Branch Code')
# merged['Alloted Program']=merged['description']
# cols= [ 'Application ID','Alloted Quota','Alloted Program', 'Alloted Institute']
# merged = merged[cols]

# merged = merged.drop_duplicates()

# merged.to_csv('/home/it/Desktop/RUN/NEUT_Official/NEUT_R1/Codes/SeatAllocationSummary1.csv', index=False)


dom['Quota']=dom['id']
merged = pd.merge(df,dom,on='Quota')
merged['Alloted Quota']=merged['description']
cols= ['Application ID', 'Alloted Quota','Category','Subcategory']
merged = merged[cols]


cat['Category']=cat['id']
merged = pd.merge(merged,cat,on='Category')
merged['CAT']=merged['description']
cols= [ 'Application ID','Alloted Quota','CAT','Subcategory']
merged = merged[cols]

merged = merged.drop_duplicates()


cols= [ 'Alloted Quota','CAT','Subcategory']
merged = merged[cols]
merged.head()

seat_allocation_summary = merged.groupby([ 'Alloted Quota','CAT','Subcategory']).size().reset_index(name='SeatsAllotted')

# seat_allocation_summary.head()

# # Group by the relevant columns and count the number of entries in each group
# seat_allocation_summary = df.groupby([ 'Alloted Quota', 'Category']).size().reset_index(name='SeatsAllotted')
# # seat_allocation_summary = df.groupby([ 'Quota', 'Category', 'Subcategory']).size().reset_index(name='SeatsAllotted')
# # seat_allocation_summary = df.groupby([ 'Quota', 'Category', 'Subcategory','Program Code','Branch Code']).size().reset_index(name='SeatsAllotted')
# # seat_allocation_summary = df.groupby(['Program Code','Branch Code']).size().reset_index(name='SeatsAllotted')
# # seat_allocation_summary = df.groupby([ 'Quota','Program Code','Branch Code']).size().reset_index(name='SeatsAllotted')




# Save the summary to a new CSV file
seat_allocation_summary.to_csv('/home/it/Desktop/RUN/NEUT_Official/NEUT_R1/Codes/SeatAllocationSummarybig.csv', index=False)


# # Display the summary
# # print(seat_allocation_summary)
# # seat_allocation_summary = seat_allocation_summary.groupby([ 'Quota']).size().reset_index(name='SeatsAllotted')
# # seat_allocation_summary.to_csv('/home/it/Desktop/RUN/NEUT_Official/NEUT_R1/Codes/SeatAllocationSummary.csv', index=False)
